# 

In [2]:
# facts
# Signatures are continuous segments of chromosomes
# Genes can (and often do) belong to multiple signatures


SyntaxError: invalid syntax (41293311.py, line 2)

In [11]:
import pandas as pd
import pathlib
import os
import json
import itertools
import numpy as np
from tqdm import tqdm

root = pathlib.Path("/data/rbg/shared/datasets/TCGA/")

In [28]:

def add_outcome(elem):
    vital_status = elem['demographic'].get('vital_status')

    if vital_status == 'Dead':
        patient_death = elem['demographic'].get('days_to_death')
        return [1, int(patient_death)]

    elif vital_status == "Alive":
        follow_ups = elem['follow_ups']
        follow_up_days = [
            f.get('days_to_follow_up')
            for f in follow_ups
            if f.get('days_to_follow_up') is not None
        ]
        diagnosis_days = [
            d.get('days_to_last_follow_up')
            for d in elem['diagnoses']
            if d.get('days_to_last_follow_up') is not None
        ]
        all_days = follow_up_days + diagnosis_days
        date_of_last_checkup = max(all_days) if all_days else None

        return [0, int(date_of_last_checkup)]
    else:
        typing.assert_never("vital status not in Alive/Dead")


def remove_sparse_features(X, threshold=0.05):
    non_zero_fraction = (X != 0).sum(axis=0) / X.shape[0]
    keep = non_zero_fraction > threshold
    return X.loc[:, keep]


def remove_perfect_correlations(X):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] == 1)]
    return X.drop(columns=to_drop)


# ids = pd.read_json("ids/tcga_cohort.json")["case_id"]
ids = pd.read_csv("sample_ids.csv")["Sample"]

case_ids_raw = pd.read_json("./clinical-tcga-brca.json").set_index("submitter_id")
case_ids = case_ids_raw.loc[ids]
case_ids["outcome"] = case_ids.apply(add_outcome, axis=1)

case_ids = case_ids.reset_index()
print(case_ids.shape)

case_ids = case_ids.drop_duplicates(subset=["case_id"])
print(case_ids.shape)

case_ids.head()


(133, 16)
(133, 16)


,submitter_id,disease_type,project,days_to_consent,diagnoses,consent_type,demographic,primary_site,updated_datetime,case_id,follow_ups,index_date,state,lost_to_followup,exposures,outcome
0,TCGA-A1-A0SP,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},55.0,"[{'tissue_or_organ_of_origin': 'Breast, NOS', ...",Informed Consent,{'demographic_id': 'f7ed8691-ed3d-54bf-94c5-7f...,Breast,2025-01-06T07:38:56.031656-06:00,a9bb8159-32f0-454c-a946-b3286a52b9d5,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[0, 584]"
1,TCGA-A2-A04P,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},22.0,"[{'morphology': '8500/3', 'submitter_id': 'TCG...",Informed Consent,{'demographic_id': '92faa22e-8e19-5502-987c-a6...,Breast,2025-01-06T07:38:16.470569-06:00,ccd4a24b-d8cc-4686-9dee-c98b0c5a8d21,"[{'days_to_progression': 102, 'timepoint_categ...",Diagnosis,released,No,NaN,"[1, 548]"
2,TCGA-A2-A04T,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},29.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': 'dd68f2a1-f279-567f-83b6-4c...,Breast,2025-01-05T21:21:42.777520-06:00,b58ad350-5140-4fa8-bc2c-24bca8395f3a,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[0, 2246]"
3,TCGA-A2-A04U,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},23.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '9145b76c-61b5-56e6-876e-58...,Breast,2025-01-05T23:51:14.306282-06:00,1c3610f7-e0aa-48d7-9a27-0dbaf6e244f9,"[{'timepoint_category': 'Follow-up', 'follow_u...",Diagnosis,released,Yes,NaN,"[0, 2654]"
4,TCGA-A2-A0CM,Ductal and Lobular Neoplasms,{'project_id': 'TCGA-BRCA'},-212.0,"[{'synchronous_malignancy': 'No', 'ajcc_pathol...",Informed Consent,{'demographic_id': '0f8728a4-3abb-5a78-84c3-3d...,Breast,2025-01-06T07:33:56.519794-06:00,eb2dbb4f-66b6-4525-8323-431970f7a64e,"[{'timepoint_category': 'Last Contact', 'follo...",Diagnosis,released,NaN,NaN,"[1, 754]"


In [5]:
dna_mapping = "/data/rbg/shared/datasets/TCGA/Genomic/copy_number.gdc_sample_sheet.2025-07-09.tsv"
caseid_copynum_mapping = pd.read_csv(dna_mapping, delimiter='\t')

caseid_copynum_mapping['Case ID'] = caseid_copynum_mapping['Case ID'].str.split(',')
# print(caseid_copynum_mapping.shape)
caseid_copynum_mapping = caseid_copynum_mapping.explode('Case ID')
caseid_copynum_mapping['Case ID'] = caseid_copynum_mapping['Case ID'].str.strip()
# print(caseid_copynum_mapping.shape)
caseid_copynum_mapping = caseid_copynum_mapping.drop_duplicates(subset=["Case ID"]) # I'm pretty sure this is valid bc I explode only on Case ID - meaning everything else is duplicated. This whole process might actualyl be unncesary. TODO: check if all "Case IDs pairs" are just twins
print(caseid_copynum_mapping.shape)

# caseid_copynum_mapping.head()

# stupid test to check whether any of hte records had duplicate sample IDs
# def foo(x):
#     a = list(map(lambda x: x.strip(), x["Sample ID"].split(",")))
#     print(a[0])
#     print(a[1])
#     # return a[0] == a[1]
# caseid_copynum_mapping = caseid_copynum_mapping[caseid_copynum_mapping.apply(foo, axis=1)]jk:w


(1845, 11)


,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Tissue Type,Tumor Descriptor,Specimen Type,Preservation Method
0,b260bc0a-bf82-4fe8-b1ca-1ba43a4d741a,TCGA-C8-A27B-01A-11D-A89A-36.WholeGenome.RP-16...,Copy Number Variation,Copy Number Segment,TCGA-BRCA,TCGA-C8-A27B,"TCGA-C8-A27B-10A, TCGA-C8-A27B-01A","Normal, Tumor","Not Applicable, Primary","Peripheral Blood NOS, Solid Tissue","Unknown, OCT"
1,8ca11193-5883-4b75-9e34-f9dcbf7040a7,TCGA-B6-A0WW-01A-11D-A893-36.WholeGenome.RP-16...,Copy Number Variation,Copy Number Segment,TCGA-BRCA,TCGA-B6-A0WW,"TCGA-B6-A0WW-01A, TCGA-B6-A0WW-10A","Tumor, Normal","Primary, Not Applicable","Solid Tissue, Peripheral Blood NOS","OCT, Unknown"
2,d9de4dd4-6dd8-4a78-901a-35251c228ec6,TCGA-E9-A1R0-01A-22D-A898-36.WholeGenome.RP-16...,Copy Number Variation,Copy Number Segment,TCGA-BRCA,TCGA-E9-A1R0,"TCGA-E9-A1R0-10A, TCGA-E9-A1R0-01A","Normal, Tumor","Not Applicable, Primary","Peripheral Blood NOS, Solid Tissue","Unknown, OCT"
3,1eb79fa2-5f92-40f7-b36f-8a9844ebedf1,66a9e6d2-6dea-4475-ab39-7b5ffa3d9faf.wgs.ASCAT...,Copy Number Variation,Gene Level Copy Number,TCGA-BRCA,TCGA-BH-A0C1,"TCGA-BH-A0C1-01B, TCGA-BH-A0C1-10A","Tumor, Normal","Primary, Not Applicable","Solid Tissue, Peripheral Blood NOS","OCT, Unknown"
5,42e20b03-cc2a-40e2-ba66-d34ca7890b96,TCGA-AC-A5EH-01A-11D-A89D-36.WholeGenome.RP-16...,Copy Number Variation,Copy Number Segment,TCGA-BRCA,TCGA-AC-A5EH,"TCGA-AC-A5EH-01A, TCGA-AC-A5EH-10A","Tumor, Normal","Primary, Not Applicable","Solid Tissue, Peripheral Blood NOS","Unknown, Unknown"


In [7]:
# add copynumbesr to "IDs" dataset
# FIXME: figure out why 21 records are being dropped - the final result should still be 133
print(case_ids.shape)
data= pd.merge(
    case_ids,
    caseid_copynum_mapping[['Case ID', 'Sample ID', 'File ID', "File Name"]],  # select only the needed column
    left_on='submitter_id',
    right_on='Case ID',
    how="inner"
)
print(data.shape)
# data.head()

(133, 16)
(112, 20)


In [16]:
import re
import pandas as pd

output = []
for idx, elem in tqdm(list(data.iterrows())):
    path = f"{root}/Genomic/downloads/{elem['File ID']}/{elem['File Name']}"
    CNAs = pd.read_csv(path, delimiter="\t")

    
    # Example
    # signatures = pd.DataFrame({"signature": [...]})
    signatures = pd.read_excel("copynum_signatures.xlsx").drop(0).rename(columns={"Supplementary Data 2. Annotation of copy number segments": "signature"})
    
    def extract_region(sig):
        # Match first chr/start-end pattern
        match = re.search(r"(chr)?(\d+|X|Y|M):(\d+)-(\d+)", sig)
        if match:
            chrom = match.group(1) + match.group(2) if match.group(1) else "chr" + match.group(2)
            start = int(match.group(3))
            end = int(match.group(4))
            return chrom, start, end
        return None, None, None
    
    signatures = pd.concat([
        signatures["signature"],
        signatures["signature"].apply( lambda x: pd.Series(extract_region(x)) )
    ], axis=1)
    
    signatures = signatures.rename(columns={0: "chromosome", 1: "start", 2: "end"})
    

    merged = CNAs.merge(
        signatures, 
        left_on='Chromosome', 
        right_on='chromosome', 
        how='left'
    )
    mask = (merged['Start'] >= merged['start'])
    
    result = merged.loc[mask, CNAs.columns.tolist() + ['signature']]

    def label_score(score):
        if score > 0.3:
            return "gain"
        elif score < -0.3:
            return "loss"
        else:
            return "no-change"
    signature_scores = result.groupby(by=["signature"])["Segment_Mean"].apply(lambda x: x.mean()).apply(label_score)

    signature_scores.loc["Case ID"] = elem["submitter_id"]
    

    output.append(signature_scores)
        
    


 31%|██████████████████████████████████████▊                                                                                     | 35/112 [00:13<00:30,  2.56it/s]


KeyError: 'Chromosome'

In [21]:
cna_data = pd.DataFrame(output).reset_index(drop=True)

cna_data.head()

signature,1.p wholearm.chr1:1-124300000,1.q wholearm.chr1:124300000-247249719,10.p wholearm.chr10:1-40300000,10.q wholearm.chr10:40300000-135374737,11.p wholearm.chr11:1-52900000,11.q wholearm.chr11:52900000-134452384,12.p wholearm.chr12:1-35400000,12.q wholearm.chr12:35400000-132349534,13.q wholearm.chr13:16000000-114142980,14.q wholearm.chr14:15600000-106368585,...,chr4:186684565-191273063.BeroukhimS2.4q35.2.del.,chr6:161612277-163134099.BeroukhimS2.6q26.del.PARK2,chr7:144118814-148066271.BeroukhimS2.7q35.del.CNTNAP2,chr7:156893473-158821424.BeroukhimS2.7q36.3.del.PTPRN2,chr9:137859478-140273252.BeroukhimS2.9q34.3.amp.,chrX:148267454-148753845.BeroukhimS5..amp.,chrX:152729030-154913754.BeroukhimS2.Xq28.amp.,chr20:56249879-56298730.BeroukhimS5..amp.,chr20:59868877-61137250.BeroukhimS5..amp.,chr20:61329497-62435964.BeroukhimS2.2q13.33.amp.
0,no-change,no-change,loss,loss,no-change,no-change,gain,no-change,loss,loss,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no-change,no-change,no-change,loss,loss,loss,no-change,loss,loss,no-change,...,loss,no-change,no-change,loss,loss,no-change,no-change,NaN,NaN,NaN
2,no-change,no-change,loss,loss,no-change,no-change,no-change,no-change,loss,loss,...,loss,no-change,no-change,gain,no-change,NaN,NaN,no-change,no-change,no-change
3,gain,gain,gain,gain,no-change,no-change,gain,gain,no-change,loss,...,gain,loss,no-change,NaN,NaN,no-change,no-change,gain,gain,no-change
4,no-change,no-change,gain,no-change,no-change,loss,no-change,no-change,loss,no-change,...,no-change,no-change,no-change,loss,loss,no-change,no-change,gain,gain,gain


In [34]:
cna_data["Case ID"]

0     TCGA-A1-A0SP
1     TCGA-A2-A04P
2     TCGA-A2-A04T
3     TCGA-A2-A04U
4     TCGA-A2-A0CM
5     TCGA-A2-A0D0
6     TCGA-A2-A0D2
7     TCGA-A2-A0ST
8     TCGA-A2-A0T0
9     TCGA-A2-A0YE
10    TCGA-A2-A0YM
11    TCGA-A2-A3XT
12    TCGA-A2-A3XU
13    TCGA-A2-A3XX
14    TCGA-A2-A3XY
15    TCGA-A7-A0DA
16    TCGA-A7-A26I
17    TCGA-A7-A4SD
18    TCGA-A7-A4SE
19    TCGA-A7-A6VV
20    TCGA-A7-A6VW
21    TCGA-A7-A6VY
22    TCGA-A8-A07O
23    TCGA-A8-A07R
24    TCGA-A8-A08R
25    TCGA-AC-A8OQ
26    TCGA-AN-A04D
27    TCGA-AN-A0AR
28    TCGA-AN-A0AT
29    TCGA-AN-A0FX
30    TCGA-AN-A0G0
31    TCGA-AN-A0XU
32    TCGA-AO-A0J6
33    TCGA-AO-A124
34    TCGA-AO-A128
Name: Case ID, dtype: object

In [42]:
case_ids = case_ids[["submitter_id", "outcome"]]

df = case_ids.merge(
    cna_data,
    right_on=["Case ID"],
    left_on=["submitter_id"],
    how="inner"
)

print(case_ids.shape)
print(cna_data.shape)
print(df.shape)

(133, 2)
(35, 532)
(35, 534)


In [50]:
# df = df.drop(["Case ID", "submitter_id"], axis=1)
# X = df.drop(["outcome"], axis=1)
# y = df["outcome"]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df:
    if col != "outcome":
        df[col] = le.fit_transform(df[col])
df.head()

,outcome,1.p wholearm.chr1:1-124300000,1.q wholearm.chr1:124300000-247249719,10.p wholearm.chr10:1-40300000,10.q wholearm.chr10:40300000-135374737,11.p wholearm.chr11:1-52900000,11.q wholearm.chr11:52900000-134452384,12.p wholearm.chr12:1-35400000,12.q wholearm.chr12:35400000-132349534,13.q wholearm.chr13:16000000-114142980,...,chr4:186684565-191273063.BeroukhimS2.4q35.2.del.,chr6:161612277-163134099.BeroukhimS2.6q26.del.PARK2,chr7:144118814-148066271.BeroukhimS2.7q35.del.CNTNAP2,chr7:156893473-158821424.BeroukhimS2.7q36.3.del.PTPRN2,chr9:137859478-140273252.BeroukhimS2.9q34.3.amp.,chrX:148267454-148753845.BeroukhimS5..amp.,chrX:152729030-154913754.BeroukhimS2.Xq28.amp.,chr20:56249879-56298730.BeroukhimS5..amp.,chr20:59868877-61137250.BeroukhimS5..amp.,chr20:61329497-62435964.BeroukhimS2.2q13.33.amp.
0,"[0, 584]",2,2,1,1,2,2,0,2,1,...,3,3,3,3,3,3,3,3,3,3
1,"[1, 548]",2,2,2,1,1,1,2,1,1,...,1,2,2,1,1,2,2,3,3,3
2,"[0, 2246]",2,2,1,1,2,2,2,2,1,...,1,2,2,0,2,3,3,2,2,2
3,"[0, 2654]",0,0,0,0,2,2,0,0,2,...,0,1,2,3,3,2,2,0,0,2
4,"[1, 754]",2,2,0,2,2,1,2,2,1,...,2,2,2,1,1,2,2,0,0,0


In [73]:
import multiprocessing
multiprocessing.cpu_count()

40

In [79]:
# resampling test
from sklearn.model_selection import ShuffleSplit, ParameterGrid
from sklearn.linear_model import LinearRegression
from sksurv.linear_model import CoxnetSurvivalAnalysis
from joblib import Parallel, delayed
import tqdm
import warnings

from sksurv.util import Surv
from sklearn.utils.validation import validate_data

l1_ratios = np.arange(0.1, 1.0, 0.1)
lambdas = np.logspace(2, -2, 20)

param_grid = list(ParameterGrid({
    'l1_ratio': l1_ratios,
    'lambda': lambdas
}))
print(len(param_grid))


cv = ShuffleSplit(n_splits=25, test_size=0.25, random_state=42)
gtmp = 0

from sksurv.metrics import NoComparablePairException

def train_parameters(cv, param):
    l1_ratio = param["l1_ratio"]
    lambda_ = param["lambda"]

    score_sum = 0
    score_count = 0

    for train_idx, test_idx in cv.split(df):
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]

        X_train = train_df.drop("outcome", axis=1).apply(pd.to_numeric, errors='raise').to_numpy()
        X_test = test_df.drop("outcome", axis=1).apply(pd.to_numeric, errors='raise').to_numpy()

        train_outcome_df = pd.DataFrame(train_df["outcome"].tolist(), columns=["event", "duration"]).astype({"event": bool, "duration": float})
        y_train = Surv.from_dataframe("event", "duration", train_outcome_df)

        test_outcome_df = pd.DataFrame(test_df["outcome"].tolist(), columns=["event", "duration"]).astype({"event": bool, "duration": float})
        y_test = Surv.from_dataframe("event", "duration", test_outcome_df)

        cox = CoxnetSurvivalAnalysis(
            l1_ratio=l1_ratio,
            alphas=np.atleast_1d(lambda_).astype(float),
            max_iter=1000
        )
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            cox.fit(X=X_train, y=y_train)

        try:
            if pd.DataFrame(y_test)["event"].any():
                score = cox.score(X_test, y_test)
                score_sum += score
                score_count += 1
            else:
                print("!!! skipped -- no uncencored records")
        except NoComparablePairException:
            print("Skipping fold - no comparable pairs in test set")

    return {
        'l1_ratio': l1_ratio,
        'lambda': lambda_,
        'score': score_sum / score_count if score_count > 0 else float('nan')
    }


gtmp = Parallel(backend="loky", n_jobs=-1, verbose=1)(delayed(train_parameters)(cv, param) for param in tqdm.tqdm(param_grid))







180




  0%|                                                                                                                                     | 0/180 [00:00<?, ?it/s]

!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
Skipping fold - no comparable pairs in test set
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
Skipping fold - no comparable pairs in test set
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
Skipping fold - no comparable pairs in test set
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
!!! skipped -- no uncencored records
Skipping fold - no comparable pairs in test set
!!! skipped -- no uncencored records
!!! skipped -- no uncencored re

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


  1%|▋                                                                                                                            | 1/180 [00:00<01:52,  1.60it/s]

 22%|███████████████████████████▌                                                                                                | 40/180 [00:06<00:21,  6.55it/s][Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    6.7s


 33%|█████████████████████████████████████████▎                                                                                  | 60/180 [00:09<00:18,  6.64it/s]

 44%|███████████████████████████████████████████████████████                                                                     | 80/180 [00:12<00:14,  6.75it/s]

 56%|████████████████████████████████████████████████████████████████████▎                                                      | 100/180 [00:15<00:11,  6.69it/s]

 67%|██████████████████████████████████

In [83]:
print(len(gtmp))
print(gtmp[0:10])

180
[{'l1_ratio': np.float64(0.1), 'lambda': np.float64(100.0), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(61.58482110660264), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(37.926901907322495), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(23.357214690901223), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(14.38449888287663), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(8.858667904100827), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(5.45559478116852), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(3.3598182862837827), 'score': np.float64(0.5)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(2.06913808111479), 'score': np.float64(0.41228070175438597)}, {'l1_ratio': np.float64(0.1), 'lambda': np.float64(1.2742749857031341), 'score': np.float64(0.45560871876661

In [85]:
best_params = max(gtmp, key=lambda x: x['score'])
l1_ratio, _lambda, score = best_params['l1_ratio'], best_params['lambda'], best_params['score']

In [94]:
X = df.drop("outcome", axis=1).apply(pd.to_numeric, errors='raise').to_numpy()

_y= pd.DataFrame(df["outcome"].tolist(), columns=["event", "duration"]).astype({"event": bool, "duration": float})
y = Surv.from_dataframe("event", "duration", _y)

model = CoxnetSurvivalAnalysis(
            l1_ratio=l1_ratio,
            alphas=np.atleast_1d(_lambda).astype(float),
            max_iter=1000
        )

model.fit(X=X, y=y)

coef_series = pd.Series(model.coef_.reshape(-1), index=df.drop("outcome", axis=1).columns)


/tmp/ipykernel_904891/1548947657.py:12: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=1000).
  model.fit(X=X, y=y)


In [102]:
coef_series[coef_series != 0].sort_values(key=lambda x: abs(x))

chr14:93528266-103946058.BeroukhimS5..amp.               0.000486
chr14:92773649-95250286.Basal.14q32.13-99.del            0.000973
chr10:89467202-90419015.BeroukhimS2.1q23.31.del.PTEN     0.001311
6.q   wholearm.chr6:60500000-170899992                   0.001596
chr7:144118814-148066271.BeroukhimS2.7q35.del.CNTNAP2    0.002943
                                                           ...   
chr20:41621022-45719023.LumB.20q13.12-166.amp           -0.174658
chr9:137859478-140273252.BeroukhimS2.9q34.3.amp.        -0.226767
chr9:130999928-132303780.BeroukhimS5..del.              -0.244604
chr8:140458177-146274826.BeroukhimS2.8q24.3.amp.        -0.331828
chr20:61329497-62435964.BeroukhimS2.2q13.33.amp.        -0.339865
Length: 118, dtype: float64